## Uso YT dlp para la demo, scrapping basico de metadatos. Quiza tengo que pasar a una api oficial para que sea mas estable o tener acceso a mas datos

In [5]:
import yt_dlp
import json
import time

lista_canales = [
    "https://www.youtube.com/@BabyBusES",
    "https://www.youtube.com/@SmileandLearnEspa%C3%B1ol",
    "https://www.youtube.com/@creativoskids",
    "https://www.youtube.com/@JuguetesyColores",
    "https://www.youtube.com/@GenevievesPlayhouse",
    "https://www.youtube.com/@GamingWithGTA00",
    "https://www.youtube.com/@videosdejuguetespawpatrole5118",
]

def scrapear_lista_canales(urls):
    ydl_opts = {
        'quiet': True,
        'extract_flat': True,  # (solo metadatos)
        'playlistend': 10,     # Límite de videos
        'ignoreerrors': True,  
        'no_warnings': True,
        # 'sleep_interval': 1,
    }

    resultados = []

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        
        for index, url in enumerate(urls):
            
            try:
                # Extraemos la info
                info = ydl.extract_info(url, download=False)
                
                # A veces yt-dlp devuelve None si la url está mal
                if not info:
                    print(f" No se pudo obtener info de {url}")
                    continue

                # Estructuramos los datos
                datos = {
                    "id_canal": info.get('id'),
                    "nombre": info.get('uploader') or info.get('title'),
                    "descripcion": info.get('description', ''),
                    "suscriptores": info.get('channel_follower_count'),
                    "url_origen": url,
                    # Juntamos los títulos de los videos en una lista simple
                    "videos_recientes": []
                }

                # Extraer videos (validando que existan entradas)
                if 'entries' in info:
                    for video in info['entries']:
                        if video and video.get('title'):
                            datos["videos_recientes"].append(video.get('title'))
                
                resultados.append(datos)
                print(f"    OK: {datos['nombre']} ({len(datos['videos_recientes'])} videos)")

            except Exception as e:
                print(f"    Error en este canal: {str(e)}")

    return resultados

# --- EJECUCIÓN ---

start = time.time()
data_scraped = scrapear_lista_canales(lista_canales)
end = time.time()

print(f"\n✨ Proceso terminado en {end - start:.2f} segundos.")

# 2. Guardar en un archivo JSON (Esto es lo que usará tu IA después)
nombre_archivo = "dataset_canales.json"
with open(nombre_archivo, 'w', encoding='utf-8') as f:
    json.dump(data_scraped, f, ensure_ascii=False, indent=4)

print(f"📂 Datos guardados en '{nombre_archivo}'.")

    OK: BabyBus - Canciones Infantiles & Videos para Niños (3 videos)
    OK: Smile and Learn - Español (3 videos)
    OK: Creativos Kids (2 videos)
    OK: Juguetes y Colores (10 videos)
    OK: Genevieve's Playhouse - Learning Videos for Kids (2 videos)
    OK: Gaming With GTA  (10 videos)
    OK: Videos de juguetes Paw Patrol en español (10 videos)

✨ Proceso terminado en 4.43 segundos.
📂 Datos guardados en 'dataset_canales.json'.


In [2]:
import json
import time
from sentence_transformers import SentenceTransformer, util

try:
    with open('dataset_canales.json', 'r', encoding='utf-8') as f:
        datos_canales = json.load(f)
    print(f"   --> {len(datos_canales)} canales cargados.")
except FileNotFoundError:
    print(" Error: No encontré el archivo 'dataset_canales.json'. Ejecuta el scraper primero.")
    exit()

# El modelo necesita un solo texto por canal para entender de qué trata.
corpus_textos = []

for canal in datos_canales:
    # Unimos los titulos de videos con comas
    txt_videos = ", ".join(canal.get('videos_recientes', []))
    
    # Creamos el texto maestro para la IA
    # "Nombre: [X]. Descripción: [Y]. Contenido reciente: [Z]"
    texto_full = f"Canal: {canal['nombre']}. Descripción: {canal['descripcion']}. Temas de videos: {txt_videos}"
    
    corpus_textos.append(texto_full)

# --- INDEXACIÓN  ---
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

embeddings_db = model.encode(corpus_textos, convert_to_tensor=True)

def buscar(query_usuario, top_k=3):
    start_time = time.time()
    
    # Vectorizamos lo que escribió el usuario
    query_embedding = model.encode(query_usuario, convert_to_tensor=True)
    
    # Buscamos similitud
    hits = util.semantic_search(query_embedding, embeddings_db, top_k=top_k)
    
    print(f" Búsqueda: '{query_usuario}' ({time.time() - start_time:.4f} seg)")
    
    for hit in hits[0]:
        score = hit['score']
        id_canal = hit['corpus_id'] # El índice en la lista original
        canal_original = datos_canales[id_canal] # Recuperamos el objeto JSON original
        
        # Filtro de calidad
        if score < 0.25: 
            continue
            
        print(f" Match: {score:.2f} | {canal_original['nombre']}")
        # por que hizo match?
        print(f"      (Contexto: {canal_original['videos_recientes'][:2]}...)")
    print("-" * 30)

# --- PRUEBAS ---
# Ahora simulamos las queries libres de tus clientes
buscar("videos")
buscar("aprender")
buscar("juegos")
buscar("tutoriales")
buscar("dibujos animados para bebes")
buscar("niños creativos")
buscar("diversion")
buscar("jugar")

   --> 7 canales cargados.


Loading weights: 100%|██████████| 199/199 [00:00<00:00, 563.98it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


 Búsqueda: 'videos' (0.0274 seg)
 Match: 0.49 | Genevieve's Playhouse - Learning Videos for Kids
      (Contexto: ["Genevieve's Playhouse - Learning Videos for Kids - Videos", "Genevieve's Playhouse - Learning Videos for Kids - Shorts"]...)
 Match: 0.49 | Videos de juguetes Paw Patrol en español
      (Contexto: ['We play and learn to brush our teeth with the puppies', 'Paw Patrol visits the aquarium 🐠✨ | Daycare']...)
 Match: 0.46 | Creativos Kids
      (Contexto: ['Creativos Kids - Videos', 'Creativos Kids - Shorts']...)
------------------------------
 Búsqueda: 'aprender' (0.0235 seg)
 Match: 0.48 | Smile and Learn - Español
      (Contexto: ['Smile and Learn - Español - Videos', 'Smile and Learn - Español - Live']...)
 Match: 0.43 | Genevieve's Playhouse - Learning Videos for Kids
      (Contexto: ["Genevieve's Playhouse - Learning Videos for Kids - Videos", "Genevieve's Playhouse - Learning Videos for Kids - Shorts"]...)
 Match: 0.37 | Juguetes y Colores
      (Contexto: ['Kaden y


#### *VOLVAMOS A INTENTAR, USANDO LA BASE DE DATOS QUE YA TENEMOS EN KIDSCORP YT*


In [2]:
import os
import pickle  # <--- Librería para guardar archivos

# Nombre del archivo donde guardaremos los "cerebros"
ARCHIVO_EMBEDDINGS = "embeddings_cache.pkl"

def cargar_o_generar_embeddings(df):
    # 1. ¿Ya existe el archivo guardado?
    if os.path.exists(ARCHIVO_EMBEDDINGS):
        print(f"💾 Cargando embeddings desde '{ARCHIVO_EMBEDDINGS}'...")
        with open(ARCHIVO_EMBEDDINGS, 'rb') as f:
            datos_guardados = pickle.load(f)
            # Verificamos que coincidan con los datos actuales (opcional pero recomendado)
            if len(datos_guardados) == len(df):
                print("✅ Embeddings cargados exitosamente.")
                return datos_guardados
            else:
                print(" Recalculando embeddings...")

    # 2. Si no existe o cambiaron los datos, generamos de cero
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    
    # Asegúrate de haber creado la columna 'texto_para_ia' antes
    embeddings = model.encode(df['texto_para_ia'].tolist(), convert_to_tensor=True)
    
    # 3. Guardamos para la próxima
    print(f"💾 Guardando en '{ARCHIVO_EMBEDDINGS}' para el futuro...")
    with open(ARCHIVO_EMBEDDINGS, 'wb') as f:
        pickle.dump(embeddings, f)
        
    return embeddings

In [4]:
import os
import psycopg2
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import time
from dotenv import load_dotenv

# Carga las variables
load_dotenv(override=True)

def obtener_datos():
    conn = None
    try:
        conn = psycopg2.connect(
            host=os.getenv("DB_HOST"),
            database=os.getenv("DB_NAME"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASS"),
            port="5432",
            sslmode="require"
        )
        
        query = """
            SELECT channel_title, channel_description, channel_bs_ch_keywords 
            FROM ods.tbl_canales 
            LIMIT 10000;
        """
        
        df = pd.read_sql_query(query, conn)
        return df

    except Exception as e:
        print(f"Error al conectar: {str(e)}")
        return pd.DataFrame()
    finally:
        if conn: conn.close()

# --- PROCESO PRINCIPAL ---
start = time.time()

# 1. Obtener Datos
df_canales = obtener_datos()

if df_canales.empty:
    print(" No se descargaron datos.")
    exit()

# Rellenamos los NULLs con texto vacío para que no explote la IA
df_canales = df_canales.fillna('')

# 3. CREAR TEXTO ENRIQUECIDO
# Juntamos Título + Descripción + Keywords en una sola columna para la IA
df_canales['texto_para_ia'] = (
    "Canal: " + df_canales['channel_title'] + ". " +
    "Descripción: " + df_canales['channel_description'] + ". " +
    "Keywords: " + df_canales['channel_bs_ch_keywords']
)

# 4. VECTORIZAR
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# 2. En lugar de llamar a model.encode directo, llamas a la función inteligente
embeddings_sb = cargar_o_generar_embeddings(df_canales)

print(f" Proceso terminado en {time.time() - start:.2f} segundos.")
print(f" Canales procesados: {len(df_canales)}")
# --- TEST DE BÚSQUEDA ---
def buscar(query_usuario):
    query_vec = model.encode(query_usuario, convert_to_tensor=True)
    hits = util.semantic_search(query_vec, embeddings_sb, top_k=100)
    
    for hit in hits[0]:
        idx = hit['corpus_id']
        info_canal = df_canales.iloc[idx]
        print(f" {hit['score']:.2f} | {info_canal['channel_title']}")

# Pruébalo aquí mismo
buscar("videos de manualidades para niños")
buscar("gameplay de minecraft")

C:\Users\Giovanni Pitta\AppData\Local\Temp\ipykernel_36000\3800796933.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
Loading weights: 100%|██████████| 199/199 [00:00<00:00, 453.12it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


💾 Cargando embeddings desde 'embeddings_cache.pkl'...
✅ Embeddings cargados exitosamente.
 Proceso terminado en 25.14 segundos.
 Canales procesados: 10000
 0.79 | KiKi-RiKi Videos Infantiles para niños
 0.74 | Juntines Planes
 0.73 | Marie Leiner
 0.72 | Bee Brinquedos
 0.72 | Video Juguetes
 0.71 | Daniel Alon.Videos educativos y entretenidos.
 0.70 | Manualidades Para Todos
 0.70 | Bluey y BurriKiKi
 0.69 | Ensinando meu filho
 0.67 | HooplaKidz Edu - Educational Videos For Kids
 0.67 | Dolly & Amigos em Português - Brasil
 0.67 | INGLÉS EnSimplesPalabras
 0.66 | Brinquedoteca da Geni
 0.66 | ¡Oh, se dañó! 
 0.66 | LOS JUEGOS INFANTILES DE OLGA
 0.65 | Betinha ARTEIRA
 0.64 | LUNAVENTURAS
 0.64 | Maria Clara y JP en español
 0.64 | Mimonona Stories
 0.63 | Flash Videos - Craft
 0.63 | guiainfantil
 0.63 | FamilyTube
 0.63 | ByGuillex
 0.63 | Chipobre
 0.62 | Dima y Coches
 0.62 | Dibujitos para ti
 0.62 | WildBrain Peques
 0.61 | Talentos en Manualidades
 0.61 | Momentos DIYvertiDOLL

In [5]:
buscar("canciones")
buscar("aprender idiomas")
buscar("poopie")
buscar("mrbeast")

 0.65 | Canciones De Amor
 0.64 | Sarah Cabello
 0.63 | Lyrics Music
 0.63 | CancionesDivertidasNiños
 0.61 | Apertei, mandei!
 0.60 | SYML
 0.60 | Timeless Music
 0.60 | Reggaeton Music
 0.59 | Encanta Música
 0.59 | Romantic Music
 0.59 | Kids Songs & Stories
 0.58 |  Kasterwey Music ec
 0.58 | AKAPELA RECORDS
 0.58 | Leord music
 0.58 | PeeWee Music
 0.58 | Mainstage
 0.57 | Julmer Music
 0.57 | Dubon愛
 0.57 | Mayré Martínez
 0.57 | Best Of Rock Music
 0.57 | Ovy On The Drums
 0.57 | Apples and Bananas Português - Canção Infantil
 0.57 | Alex Boye
 0.57 | Mental music ent
 0.56 | Secplay
 0.56 | 7k7music
 0.56 | REGGAE MUSIC
 0.56 | Dream Music
 0.56 | Love Songs Collection
 0.56 | Melissa Music
 0.56 | XTRAKY MUSIC
 0.56 | Lyrics Corner
 0.56 | Charlie Parra del Riego
 0.56 | LEFUL MUSIC
 0.56 | Amasing Music
 0.56 | Mix of Indie Mainstream Music
 0.56 | GALÁXIA Music
 0.56 | Yare Music
 0.55 | Music L.A
 0.55 | Radio Rock Music
 0.55 | Tyana Music
 0.55 | Nogastic Music
 0.55 | Lo

### La base de datos vectorial eventualmente se puede guardar en postgreSQL, aunque la base de datos necesita tener instalada la extension *vector*

### Proximos pasos :
#### Mejorar la recopilacion de data. Usar la base de datos que tenemos, y descargar descripciones ultimos 10 videos y si podemos integrar el trafico de cada canal al sistema. Osea, cuantos seguidores tiene (mas asincronico) y cuanto trafico relativo tuvo en el ultimo tiempo asi nos aseguramos de que tengan un buen potencial de generacion de impresiones.
#### Convertir en clases y modularizar.
#### Pensar algun metodo de benchmark.
#### Investigar que hace la funcion semantic search (seguro es cosine similarity pero quiza hay una variante mas util para este problema).
#### Determinar modos de busqueda, osea si van a ser una palabra solo, varias palabras, ambas. Ver lo de la negativa (como hago para notBuscar, funcion aparte o puedo hacerlo todo junto?)(no quiero videos de futbol, por ejemplo)


<span style="color:green">Notas anteriores:</span>

<span style="color:white">Podria hacer un funnel con embeddings - ReRankers, pero creo que para este caso no es muy necesario (se usa para evitar fallas en entender el contexto, el reranker "lee la frase completa")

Quiza pifia con nombre inventados, por ejemplo si alguien le pinta buscar mrbeast puede no salir (solucion hibrida con un tf-idf, para resultados mas exactos)</span>
#### Definir estructura de datos
#### Integrar el grafo que tenemos (?)
#### Necesito tener una cuenta en hugging face para que el modelo pueda administrar mas rapido el trafico

In [19]:
## Codigo para la negativa, aritmetica de vectores
import numpy as np
from sentence_transformers import util
def buscar_con_negativa(query_positiva,query_negativa=None):
    vec_pos = model.encode(query_positiva, convert_to_tensor=True)
    vector_final = vec_pos

    if query_negativa:
        vec_neg = model.encode(query_negativa, convert_to_tensor=True)
        vector_final = vec_pos - (vec_neg * 0.5) ### 0.5 actua como hyperparametro, se puede retocar para mejores resultados

    hits = util.semantic_search(vector_final, embeddings_sb, top_k=50)
    for hit in hits[0]:
        idx = hit['corpus_id']
        titulo = df_canales.iloc[idx]['channel_title']
        desc = df_canales.iloc[idx]['channel_description'][:60] 
        print(f" {hit['score']:.3f} | {titulo} ({desc}...)")
buscar_con_negativa("peppa pig")
print("--- BUSQUEDA CON NEGATIVA ---")
buscar_con_negativa("peppa pig","goat")

    # Esto tambien se puede hacer con un filtro mas duro, como un if que me saque los resultados que contengan cierta palabra

 0.799 | Papai e Jujuba Games (Canal sobre o desenho Peppa Pig. Peppa é uma porquinha recho...)
 0.782 | Peppaventuras (¡Hola! :)  ¡Bienvenidos a nuestro canal de Peppa Pig! Tenemo...)
 0.751 | Canal Famiglia ("Se na sua casa você já ouviu em algum momento ""eu quero a ...)
 0.745 | CanalKids (Peppa Pig Em Português BrasilPeppa é uma porquinha famosa qu...)
 0.732 | Peppa Pig em Português Brasil - Canal Oficial (Bem-vindos ao canal oficial da Peppa Pig em Português (Brasi...)
 0.725 | Cartoon Kids - Português Brasil (Bem-vindos ao canal oficial da Peppa Pig em Português (Brasi...)
 0.723 | Peppa Juegos (Diviértete con las mejores historias de Peppa Pig, en este c...)
 0.707 | Aventuras de cerdo en español (Hola chicos ¡Nos encanta Pig Peppa y sus amigos! ¡Y nos enca...)
 0.700 | Peppa Pig Español - Canal Oficial (¡Bienvenidos al canal oficial de Peppa Pig en Castellano en ...)
 0.658 | Adivinha Kids (Neste canal você irá se divertir com os personagens do desen...)
 0.645 | CartoonKiDs 

### Probar con los clusters ya hechos, como metodo de benchmark

In [ ]:
### Vuelvo a hacer loa embeddings, pero con los datos clasificados
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import time

# --- CONFIGURACIÓN (Edita esto con tus datos reales) ---
NOMBRE_ARCHIVO = "Clasificacion/Clasificacion canales - Canales.csv"  # <--- Tu archivo Excel

# Nombres exactos de las columnas en tu Excel
COL_CATEGORIA = "Categoria"
COL_GRUPO = "ID Grupo"         # La columna que usaremos para agrupar/contar
COL_TITULO = "channel_title"           # Título del canal/video
COL_DESC = "channel_description"       # Descripción (si tienes)

TOP_K_BENCHMARK = 400 

def cargar_excel():
    print(f"📂 Cargando archivo '{NOMBRE_ARCHIVO}'...")
    try:
        df = pd.read_csv(NOMBRE_ARCHIVO)
        df.fillna('Sin Categoría', inplace=True) # Rellenamos vacíos
        
        # Creamos el texto para la IA
        df['texto_para_ia'] = (
            "Título: " + df[COL_TITULO].astype(str) + ". " +
            "Descripción: " + df[COL_DESC].astype(str)
        )
        print(f"✅ Datos cargados: {len(df)} filas.")
        return df
    except Exception as e:
        print(f"❌ Error: {e}")
        exit()

# --- PREPARACIÓN ---
df_benchmark = cargar_excel()
print("🧠 Vectorizando datos (Embedding)...")
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
embeddings_db = cargar_o_generar_embeddings(df_benchmark)
print("⚡ Listo para buscar.\n")


# --- FUNCIÓN DE REPORTE ---
def reporte_categorias(query):
    print(f"\n🔎 BUSCANDO: '{query}'")
    print(f"   (Analizando los top {TOP_K_BENCHMARK} resultados más relevantes)")
    print("=" * 70)
    
    # 1. Búsqueda Semántica
    query_vec = model.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(query_vec, embeddings_db, top_k=TOP_K_BENCHMARK)
    
    # 2. Recuperar las categorías de los resultados
    indices = [hit['corpus_id'] for hit in hits[0]]
    df_resultados = df_benchmark.iloc[indices]
    
    # 3. CÁLCULO ESTADÍSTICO (Aquí está la magia) 📊
    # Contamos cuántos hay de cada categoría
    conteo = df_resultados[COL_CATEGORIA].value_counts()
    total_encontrados = len(df_resultados)
    
    # 4. IMPRIMIR TABLA
    print(f"{'CATEGORÍA':<25} | {'CANT':<5} | {'% DEL TOTAL':<12}")
    print("-" * 70)
    
    for categoria, cantidad in conteo.items():
        porcentaje = (cantidad / total_encontrados) * 100
        
        print(f"{str(categoria):<25} | {cantidad:<5} | {porcentaje:>6.1f}%")
        
    print("-" * 70)
    
    # (Opcional) Mostramos el Top 3 de ejemplos concretos para validar
    print("EJEMPLOS TOP 3 ENCONTRADOS:")
    for i in range(min(3, len(df_resultados))):
        fila = df_resultados.iloc[i]
        print(f" - [{fila[COL_CATEGORIA]}] {fila[COL_TITULO]}")

# --- TUS PRUEBAS ---
reporte_categorias("Entertainment: Anime")

📂 Cargando archivo 'Clasificacion/Clasificacion canales - Canales.csv'...
✅ Datos cargados: 9042 filas.
🧠 Vectorizando datos (Embedding)...


Loading weights: 100%|██████████| 199/199 [00:00<00:00, 766.15it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Loading weights: 100%|██████████| 199/199 [00:00<00:00, 680.54it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


💾 Guardando en 'embeddings_cache.pkl' para el futuro...
⚡ Listo para buscar.


🔎 BUSCANDO: 'Entertainment: Anime'
   (Analizando los top 400 resultados más relevantes)
CATEGORÍA                 | CANT  | % DEL TOTAL  | BARRA VISUAL
----------------------------------------------------------------------
Entertainment: Anime      | 169   |   42.2%
Sin Categoría             | 87    |   21.8%
Gaming: Roblox            | 24    |    6.0%
Entertainment: Movies para teens (13-18) | 19    |    4.8%
Entertainment: Movies para tweens (10-12) | 13    |    3.2%
Gaming: general           | 10    |    2.5%
Bloopers & funny video & Challenges, Entertainment: Anime | 10    |    2.5%
Entertainment: Music for preschoolers | 8     |    2.0%
Toys: dolls, Toys: cars & trucks, Arts & Crafts: drawing & painting, Entertainment:  Shows para niños preschoolers | 7     |    1.8%
Gaming: FREE FIRE         | 6     |    1.5%
Arts & Crafts: drawing & painting | 6     |    1.5%
Entertainment: Movies para kids (7-9) | 6

### Filtrar los que son sin categoria (quiza) y los que no son viables (hay una columna que los marca como tal)

### Pruebo APPS

In [11]:
import json
import pandas as pd
import os

# --- CONFIGURACIÓN ---
ARCHIVO_ENTRADA = "inventario_full_reviews.json"
ARCHIVO_SALIDA = "inventario_sin_duplicados.json"

def limpiar_duplicados():
    print(f"📂 Cargando '{ARCHIVO_ENTRADA}'...")
    
    if not os.path.exists(ARCHIVO_ENTRADA):
        print("❌ Error: No encuentro el archivo.")
        return

    try:
        # 1. Cargar datos
        with open(ARCHIVO_ENTRADA, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # Convertir a DataFrame
        df = pd.DataFrame(data)
        total_inicial = len(df)
        print(f"   📊 Total inicial de registros: {total_inicial}")

        # 2. ELIMINAR DUPLICADOS (La Magia) ✨
        # subset='id': Solo mira la columna 'id' para decidir si es repetido.
        # keep='first': Se queda con el primero que encontró y borra los demás.
        df_limpio = df.drop_duplicates(subset='id', keep='first')
        
        # 3. Reporte
        total_final = len(df_limpio)
        borrados = total_inicial - total_final
        
        print(f"   🗑️  Se eliminaron {borrados} duplicados.")
        print(f"   ✅ Total final de apps únicas: {total_final}")

        # 4. Guardar
        # Orient='records' convierte el DataFrame de vuelta a lista de diccionarios JSON
        lista_final = df_limpio.to_dict(orient='records')
        
        with open(ARCHIVO_SALIDA, 'w', encoding='utf-8') as f:
            json.dump(lista_final, f, ensure_ascii=False, indent=4)
            
        print(f"💾 Archivo limpio guardado como: '{ARCHIVO_SALIDA}'")

    except Exception as e:
        print(f"❌ Ocurrió un error: {e}")

if __name__ == "__main__":
    limpiar_duplicados()

📂 Cargando 'inventario_full_reviews.json'...
   📊 Total inicial de registros: 2567
   🗑️  Se eliminaron 1264 duplicados.
   ✅ Total final de apps únicas: 1303
💾 Archivo limpio guardado como: 'inventario_sin_duplicados.json'


In [12]:
import json
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import time
import os

# --- CONFIGURACIÓN ---
ARCHIVO_JSON = "inventario_sin_duplicados.json"
MODELO_NOMBRE = 'paraphrase-multilingual-MiniLM-L12-v2'

def cargar_datos_json():
    print(f"📂 Cargando {ARCHIVO_JSON}...")
    
    if not os.path.exists(ARCHIVO_JSON):
        print("❌ Error: No encuentro el archivo JSON.")
        return pd.DataFrame()

    with open(ARCHIVO_JSON, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Convertimos la lista de diccionarios a DataFrame de Pandas
    df = pd.DataFrame(data)
    
    # --- LIMPIEZA CRÍTICA ---
    # 1. Rellenar vacíos con texto vacío para que no explote
    df = df.fillna('')
    
    # 2. Preferencia de Descripción:
    # A veces 'desc_larga' viene vacía pero 'desc_corta' tiene algo.
    # Creamos una columna 'descripcion_final' que use la larga, y si no hay, la corta.
    df['descripcion_final'] = df.apply(
        lambda row: row['desc_larga'] if len(row['desc_larga']) > 10 else row['desc_corta'], 
        axis=1
    )

    # 3. CREAR TEXTO RICO PARA LA IA 🧠
    # Juntamos Título + Género + Descripción
    print("⚙️  Creando textos para el modelo...")
    df['texto_para_ia'] = (
        "App: " + df['titulo_store'] + ". " +
        "Género: " + df['genero'] + ". " +
        "Descripción: " + df['descripcion_final']
    )
    
    # Filtramos las que no tengan título (basura del scraping)
    df = df[df['texto_para_ia'].str.len() > 20]
    
    print(f"✅ Datos listos: {len(df)} apps cargadas.")
    return df

# --- PROCESO PRINCIPAL ---
start = time.time()

# 1. Cargar
df_apps = cargar_datos_json()

if df_apps.empty:
    exit()

# 2. Vectorizar (Crear Embeddings)
print(f"⚡ Generando Embeddings con {MODELO_NOMBRE}...")
model = SentenceTransformer(MODELO_NOMBRE)
embeddings = cargar_o_generar_embeddings(df_apps)


print(f"⏱️ Tiempo total de preparación: {time.time() - start:.2f} segs")

# --- FUNCIÓN DE BÚSQUEDA ---
def buscar_app(query_usuario, top_k=5):
    """
    Busca apps y DEVUELVE una lista de diccionarios.
    """
    print(f"\n🔎 Buscando: '{query_usuario}'")
    
    query_vec = model.encode(query_usuario, convert_to_tensor=True)
    hits = util.semantic_search(query_vec, embeddings, top_k=top_k)
    
    resultados_encontrados = []
    
    for hit in hits[0]:
        idx = hit['corpus_id']
        app_data = df_apps.iloc[idx]
        score = hit['score']
        
        # Recuperamos datos manejando posibles errores
        titulo = app_data.get('titulo_store') or app_data.get('titulo') or "Sin Título"
        desc = app_data.get('desc_corta') or app_data.get('resumen') or ""
        
        resultados_encontrados.append({
            "titulo": titulo,
            "score": score,
            "genero": app_data.get('genero'),
            "descripcion": desc
        })
        
        # Imprimimos bonito
        print(f"   ★ {score:.3f} | {titulo}")

    # AQUI ESTÁ EL CAMBIO: Retornamos la lista
    return resultados_encontrados




📂 Cargando inventario_sin_duplicados.json...
⚙️  Creando textos para el modelo...
✅ Datos listos: 1303 apps cargadas.
⚡ Generando Embeddings con paraphrase-multilingual-MiniLM-L12-v2...


Loading weights: 100%|██████████| 199/199 [00:00<00:00, 337.83it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Loading weights: 100%|██████████| 199/199 [00:00<00:00, 380.77it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


💾 Guardando en 'embeddings_cache.pkl' para el futuro...
⏱️ Tiempo total de preparación: 56.55 segs


In [16]:
print(buscar_app("construir casas y defender y destruir"))


🔎 Buscando: 'construir casas y defender y destruir'
   ★ 0.430 | Siege Castles
   ★ 0.408 | Monster House
   ★ 0.407 | Build It: Craft Сonstruction
   ★ 0.404 | Fake Island: Demolish!
   ★ 0.401 | EM 2 - Simulador de Presidente
[{'titulo': 'Siege Castles', 'score': 0.430354505777359, 'genero': 'Action', 'descripcion': 'Build your kingdom on your island and raid other kings with your best tactics'}, {'titulo': 'Monster House', 'score': 0.4080343246459961, 'genero': 'Arcade', 'descripcion': 'GROW POWERFUL MONSTERS!'}, {'titulo': 'Build It: Craft Сonstruction', 'score': 0.40668535232543945, 'genero': 'Simulation', 'descripcion': 'Fun building game and simulator. Become a pro at crafting and building houses!'}, {'titulo': 'Fake Island: Demolish!', 'score': 0.40449726581573486, 'genero': 'Simulation', 'descripcion': 'Set the bombs, demolish buildings, skyscrapers, towers, castles. Smash Cities!'}, {'titulo': 'EM 2 - Simulador de Presidente', 'score': 0.4005656838417053, 'genero': 'Strategy

### Subdividir en 4 bases vectoriales distintas? para 4 rangos etarios
### Como filtro por idioma en la base de datos?